In [1]:
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality

# Text example

An example of creating a language model and then transfering to a classifier.

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

Open and view the independent and dependent variables:

In [3]:
df = pd.read_csv(path/'train.csv', header=None)
df.head()

,0,1
0,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,1,This is a extremely well-made film. The acting...
2,0,Every once in a long while a movie will come a...
3,1,Name just says it all. I watched this movie wi...
4,0,This movie succeeds at being one of the most u...


In [4]:
classes = read_classes(path/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

Create a `DataBunch` for each of the language model and the classifier:

In [8]:
data_lm = TextLMDataBunch.from_csv(path)
data_clas = TextClasDataBunch.from_csv(path, vocab=data_lm.train_ds.vocab)

[fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download.

In [7]:
URLs.download_wt103_model()

We'll fine-tune the language model:

In [15]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 00:44
epoch  train loss  valid loss  accuracy
1      4.924300    4.156579    0.244940  (00:22)
2      4.643353    4.090463    0.252844  (00:21)



In [16]:
learn.loss_func

<function torch.nn.functional.cross_entropy(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

Save our language model's encoder:

In [9]:
learn.save_encoder('enc')

Fine tune it to create a classifier:

In [11]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder('enc')
learn.fit(3, 1e-3)

Total time: 00:46
epoch  train loss  valid loss  accuracy
1      0.678165    0.674127    0.655000  (00:15)
2      0.655561    0.621566    0.675000  (00:15)
3      0.643573    0.590920    0.720000  (00:15)

